# Stress Assignment to Words in Corpora

To begin data analysis, each word transcribed in the [Corpus of Bay Area Spanish (CBAS)](https://spanish-portuguese.berkeley.edu/people/justin-davidson/) and the [Corpus DIMEx100](https://turing.iimas.unam.mx/~luis/DIME/CORPUS-DIMEX.html) will be analyzed according to stress, where each syllable (and corresponding vowels) will be labeled as `stressed` or `unstressed`.